In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "lastmass/Qwen3_Medical_GRPO"
device = "cpu"  # force CPU to avoid MPS OOM
dtype = torch.float32

tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=dtype,
    trust_remote_code=True,
).to(device).eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:37<00:00, 18.59s/it]


In [3]:

messages = [
    {"role": "system", "content": "You are a helpful medical assistant. Always respond in English."},
    {"role": "user", "content": "What causes anemia?"}
]
text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
inputs = tok([text], return_tensors="pt")

with torch.inference_mode():
    out = model.generate(**inputs, max_new_tokens=128, pad_token_id=tok.eos_token_id, use_cache=False)

print(tok.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip())


Anemia is a condition where your body doesn't have enough healthy red blood cells to carry oxygen to your tissues. There are several reasons why this can happen. Let's break it down.

First, let's talk about iron deficiency. Iron is crucial for making hemoglobin, the protein in red blood cells that carries oxygen. If you don't get enough iron, your body can't make enough hemoglobin, and that leads to anemia. Common causes of iron deficiency include not eating enough iron-rich foods, like red meat, beans, and leafy greens, or having a condition that makes it hard for your body to absorb iron,


In [4]:
save_dir = "./qwen3-medical"

tok.save_pretrained(save_dir)
model.save_pretrained(save_dir)